In [ ]:
import requests
from tqdm.notebook import tqdm
from datetime import datetime as dt
import datetime
import zlib
import pickle
import json

In [ ]:
def gen_amount():
    return list(range(5000,3100000,500000))

In [ ]:
def gen_rate():
    return [x * 0.5 for x in range(0,18)]

In [ ]:
def gen_periods():
    return list(range(0,62,5))

In [ ]:
def gen_date():
    date = "01.01.2022"
    format_date = "%d.%m.%Y"
    arr = []
    arr.append(date)
    for _ in range(30):
        date = dt.strptime(date, format_date) + datetime.timedelta(days=1)
        date = dt.strftime(date,"%d.%m.%Y")
        arr.append(date)
    return arr

In [ ]:
len(gen_date())*len(gen_amount())*len(gen_rate())*len(gen_periods())

In [ ]:
test_cases = []
cnt = 0
for d in tqdm(gen_date()):
    for p in gen_periods():
        for r in gen_rate():
            for a in gen_amount():
                cnt +=1
                request = {"date":d,"amount":a,"rate":r,"periods":p}
                resp = requests.post("http://localhost:8081/deposit",json=request)
                response = {"code":resp.status_code,"date":None,"sum":None,"sum_s":None,"sum_e":None}
                if resp.status_code == 200:
                    res = resp.json()
                    response["sum"] = round(sum(res.values()))
                    response["date"] = [dt.strftime(d,"%d.%m.%Y") for d in sorted(dt.strptime(row,"%d.%m.%Y") for row in res.keys())]
                    response["sum_s"] = round(res[response["date"][0]])
                    response["sum_e"] = round(res[response["date"][-1]])
                case = {"request":request, "response":response,"id":cnt}
                test_cases.append(case)     

In [ ]:
len(test_cases)

In [ ]:
data = json.dumps(test_cases)

In [ ]:
cdata = zlib.compress(data.encode(), level=-1)

In [ ]:
with open('cases.pkl', 'wb') as f:
    pickle.dump(cdata, f)